In [ ]:
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI, RateLimitError
import os

load_dotenv()

deployment_name='gpt4Test'
max_tokens=4096

client = AzureOpenAI(
    api_key=os.getenv("AZURE_API_KEY"),  
    api_version=os.getenv("AZURE_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_ENDPOINT")
)

def generate_chat_gpt_query(prompt):
    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=0,
        n=1,
        stop=None
    )

    if response.choices:
        return response.choices[0].message.content.strip()
    else:
        return "No suggestion generated."

df = pd.read_csv("action_points.csv")

context = df.to_json(orient="records")

query = (
    "Given a context containing action points from different sprint retrospectives grouped by days. Please analyze action points and provide:\n"
    "3 the most important problems action points created for.\n"
    "\tAnswer should be in the following format:\n"
    "\t**Problem**\n"
    "\tList of all action points created with intention to resolve this problem in format: ```day``` - ```action point```\n"
    "\tYour conclusion about the problem in format: \n"
    "\t\tEnumerate action points with semanticly similar content or with similar meaning met during different retrospectives. I want to identify persistance of the same problem during multiple sprints. If there are **no such action points**, **omit** this part. Do it in format:\n"
    "\t\t\t**Subtopic name of the problem found similar action points are about**"
    "\t\t\t```day``` - ```action point``` each from new line\n"
    "\t\tExpress your conclusion on this problem."
)

prompt = "Context:\n" + context + "\n\nPrompt:\n" + query
print(prompt)
ai_response = generate_chat_gpt_query(prompt)
print(ai_response)
